# **Machine Learning - Lab-Project**

---

Lab-Project of **Group X**.

**Members:**
- Member 1 <br>
- Member 2 <br>
- Member 3 <br>
- Member 4 <br>
- Member 5 <br>

---

## **//TODO ADD Lab-Project Topic**


#### //TODO Explain Lab-Project topic, as well as goal of Lab-Project.

---

## **1. Imports and Random**

---

#### 1.1 Adding the neccessary imports:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Further imports (sklearn specific)

#### 1.2 Definition of Random-Seed:

In [2]:
RSEED = np.random.seed(42)

## **2. Data Visualization & Preperation**

---

#### Maybe add something along those lines here

We have found this super cool and fancy dataset here: [Click-Me](https://www.kaggle.com/)

We decided to use this dataset for reason xyz.

#### 2.1 Importing the data into a dataframe:

In [3]:
# Import the data and visualize the first few entries.

#### 2.2 Cleaning the data: